In [79]:

import pandas as pd
import fastf1 as ff1
import fastf1.plotting
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import warnings
import re
import datetime
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.display.max_rows = 75
pd.options.display.max_columns = 99
fastf1.Cache.enable_cache('../cache')

In [80]:
gp_round = 2
url = f'https://f1-jedha-bucket.s3.eu-west-3.amazonaws.com/data/full_data-round_{gp_round}.csv'
dict_nb_laps = {1: 57, 2: 50, 3: 58, 4: 63, 5: 57, 6: 66, 7: 78, 8: 51, 9: 70, 10: 52, 11: 71, 12: 53, 13: 70, 14: 44, 15: 72, 16: 53, 17: 61, 18: 53, 19: 56, 20: 71, 21: 71, 22: 58}
nb_laps = dict_nb_laps[gp_round]

In [86]:
csv_name = f'full_data-round_{gp_round}.csv'
dffp = pd.read_csv('/home/guillaume/Python_Projects/Jedha_F1_Project/data/' + csv_name, index_col=0)
dffp

,Time,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,LapStartTime,Team,Driver,TrackStatus,IsAccurate,LapStartDate,Session
1,0 days 00:19:49.757000,1,0 days 00:01:34.167000,2,1,NaN,NaN,0 days 00:00:35.003000,0 days 00:00:29.904000,0 days 00:00:29.260000,0 days 00:18:50.593000,0 days 00:19:20.497000,0 days 00:19:49.757000,277.0,321.0,294.0,327.0,False,HARD,2.0,True,0 days 00:18:15.590000,Red Bull Racing,VER,1,True,2022-03-25 14:03:17.023000,Practice 1
3,0 days 00:23:19.557000,1,0 days 00:01:31.885000,4,1,NaN,NaN,0 days 00:00:33.945000,0 days 00:00:29.226000,0 days 00:00:28.714000,0 days 00:22:21.617000,0 days 00:22:50.843000,0 days 00:23:19.557000,279.0,327.0,297.0,331.0,False,HARD,4.0,True,0 days 00:21:47.672000,Red Bull Racing,VER,1,True,2022-03-25 14:06:49.105000,Practice 1
5,0 days 00:27:10.263000,1,0 days 00:01:31.298000,6,1,NaN,NaN,0 days 00:00:33.825000,0 days 00:00:29.070000,0 days 00:00:28.403000,0 days 00:26:12.790000,0 days 00:26:41.860000,0 days 00:27:10.263000,279.0,325.0,296.0,330.0,False,HARD,6.0,True,0 days 00:25:38.965000,Red Bull Racing,VER,1,True,2022-03-25 14:10:40.398000,Practice 1
7,0 days 00:30:59.102000,1,0 days 00:01:30.888000,8,1,NaN,NaN,0 days 00:00:33.507000,0 days 00:00:28.980000,0 days 00:00:28.401000,0 days 00:30:01.721000,0 days 00:30:30.701000,0 days 00:30:59.102000,282.0,324.0,297.0,329.0,True,HARD,8.0,True,0 days 00:29:28.214000,Red Bull Racing,VER,1,True,2022-03-25 14:14:29.647000,Practice 1
14,0 days 01:04:01.265000,1,0 days 00:01:36.267000,15,3,NaN,NaN,0 days 00:00:35.633000,0 days 00:00:30.721000,0 days 00:00:29.913000,0 days 01:03:00.631000,0 days 01:03:31.352000,0 days 01:04:01.265000,276.0,299.0,281.0,305.0,False,SOFT,7.0,False,0 days 01:02:24.998000,Red Bull Racing,VER,1,True,2022-03-25 14:47:26.431000,Practice 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,0 days 00:53:13.399000,77,0 days 00:01:30.710000,10,4,NaN,NaN,0 days 00:00:33.099000,0 days 00:00:28.988000,0 days 00:00:28.623000,0 days 00:52:15.788000,0 days 00:52:44.776000,0 days 00:53:13.399000,279.0,322.0,291.0,325.0,False,SOFT,3.0,True,0 days 00:51:42.689000,Alfa Romeo,BOT,2,True,2022-03-26 14:36:42.698000,Practice 3
331,0 days 00:56:45.821000,77,0 days 00:01:30.030000,12,4,NaN,NaN,0 days 00:00:32.920000,0 days 00:00:28.811000,0 days 00:00:28.299000,0 days 00:55:48.711000,0 days 00:56:17.522000,0 days 00:56:45.821000,283.0,327.0,296.0,329.0,True,SOFT,5.0,True,0 days 00:55:15.791000,Alfa Romeo,BOT,1,True,2022-03-26 14:40:15.800000,Practice 3
333,0 days 01:00:31.074000,77,0 days 00:01:30.158000,14,4,NaN,NaN,0 days 00:00:32.951000,0 days 00:00:28.900000,0 days 00:00:28.307000,0 days 00:59:33.867000,0 days 01:00:02.767000,0 days 01:00:31.074000,282.0,324.0,296.0,328.0,False,SOFT,7.0,True,0 days 00:59:00.916000,Alfa Romeo,BOT,1,True,2022-03-26 14:44:00.925000,Practice 3
337,0 days 01:13:48.102000,77,0 days 00:01:34.927000,18,6,NaN,NaN,0 days 00:00:35.084000,0 days 00:00:30.214000,0 days 00:00:29.629000,0 days 01:12:48.259000,0 days 01:13:18.473000,0 days 01:13:48.102000,275.0,302.0,279.0,306.0,False,SOFT,11.0,False,0 days 01:12:13.175000,Alfa Romeo,BOT,1,True,2022-03-26 14:57:13.184000,Practice 3


In [82]:
dffp = dffp.drop(dffp.columns[[0,5,6,7,8,9,10,11,12,13,14,15,16,17,21,25,26]], axis=1)

In [83]:
dffp_temp = dffp.copy()
dffp_temp['LapTime'] = dffp_temp['LapTime'].apply(pd.to_timedelta)
dffp_times = dffp_temp.groupby('Compound')['LapTime'].mean()
dffp_times = pd.DataFrame(dffp_times).sort_values(by='LapTime').reset_index()
dffp_times = dffp_times[dffp_times['Compound'] != 'UNKNOWN']
dffp_times

,Compound,LapTime
1,SOFT,0 days 00:01:31.675071428


In [84]:
hard_to_soft = (dffp_times['LapTime'].iloc[2] - dffp_times['LapTime'].iloc[0]) * 0.75
hard_to_soft

IndexError: single positional indexer is out-of-bounds

In [ ]:
medium_to_soft = (dffp_times['LapTime'].iloc[1] - dffp_times['LapTime'].iloc[0]) * 0.75
medium_to_soft

Timedelta('0 days 00:00:01.106772785')

In [ ]:
dffp = dffp[dffp['TyreLife'].notna()]

In [ ]:
dffp_copy = dffp.copy()
dffp_copy['LapTime'] = dffp['LapTime'].apply(pd.to_timedelta)

In [ ]:
def time_to_seconds(string):
    array = re.findall(r'[0-9]+', str(string))
    array = array[1:]
    array[0] = int(array[0]) * 3600
    array[1] = int(array[1]) * 60
    try : 
        array = float(str(array[0] + array[1] + int(array[2])) + "." + array[3])
        return array
    except :
        return float(str(int(array[-2]) + int(array[-1])) + ".0")

dffp_copy["LapTimeSeconds"] = dffp_copy["LapTime"].apply(time_to_seconds)


In [ ]:
compound_colors = {
    'SOFT': '#FF3333',
    'MEDIUM': '#FFF200',
    'HARD': '#EBEBEB',
}

In [ ]:
dffp_full = dffp_copy.groupby(['TyreLife','Compound'] ,as_index=False)[['LapTimeSeconds']].mean().sort_values(by= ['Compound','TyreLife'])
dffp_full

,TyreLife,Compound,LapTimeSeconds
1,2.0,HARD,94.496833
5,3.0,HARD,95.533833
8,4.0,HARD,93.742625
11,5.0,HARD,94.382000
14,6.0,HARD,93.162500
17,7.0,HARD,93.039400
20,8.0,HARD,93.246600
23,9.0,HARD,93.703200
26,10.0,HARD,95.039000
29,11.0,HARD,94.772667


In [ ]:
# dffp_filtered = dffp_full.sort_values(by= ['Compound','TyreLife'])
dffp_filtered = dffp_full[dffp_full['TyreLife'] > 5.0]
dffp_filtered

,TyreLife,Compound,LapTimeSeconds
14,6.0,HARD,93.162500
17,7.0,HARD,93.039400
20,8.0,HARD,93.246600
23,9.0,HARD,93.703200
26,10.0,HARD,95.039000
29,11.0,HARD,94.772667
32,12.0,HARD,95.681500
35,13.0,HARD,95.189667
38,14.0,HARD,95.804600
41,15.0,HARD,95.854000


In [ ]:
dffp_soft = dffp_filtered[dffp_filtered['Compound'] == 'SOFT'].reset_index(drop=True)
dffp_medium = dffp_filtered[dffp_filtered['Compound'] == 'MEDIUM'].reset_index(drop=True)
dffp_hard = dffp_filtered[dffp_filtered['Compound'] == 'HARD'].reset_index(drop=True)

In [ ]:
dffp_soft['LapTimeSeconds'].min()

91.7273

In [ ]:
dffp_soft['LapTimeSeconds'].iloc[-1] - dffp_soft['LapTimeSeconds'].iloc[0]

3.8706999999999994

In [ ]:
diff_soft = (dffp_soft['LapTimeSeconds'].iloc[-1] - dffp_soft['LapTimeSeconds'].iloc[0])
diff_medium = (dffp_medium['LapTimeSeconds'].iloc[-1] - dffp_medium['LapTimeSeconds'].iloc[0])
diff_hard = dffp_hard['LapTimeSeconds'].iloc[-1] - dffp_hard['LapTimeSeconds'].iloc[0]

In [ ]:
if 'WET' in dffp_filtered['Compound'].tolist() or 'INTERDMEDIATE' in dffp_filtered['Compound'].tolist() or diff_soft < 0 or diff_medium < 0 or diff_hard < 0:
    diff_soft_per_lap = 0.25
    diff_medium_per_lap = 0.15
    diff_hard_per_lap = 0.09
    
else:
    diff_soft_per_lap = diff_soft / len(dffp_soft)
    diff_medium_per_lap = diff_medium / len(dffp_medium)
    diff_hard_per_lap = diff_hard / len(dffp_hard)

In [ ]:
print(diff_soft_per_lap)
print(diff_medium_per_lap)
print(diff_hard_per_lap)

0.35188181818181813
0.2704880952380953
0.2199230769230776


In [ ]:
fuel_per_lap_kg = 105 / nb_laps
mean_laps = 60
time_gain_per_kg = 0.035
laps_coeff = mean_laps / nb_laps
time_gained_per_lap = laps_coeff * time_gain_per_kg * fuel_per_lap_kg
time_gained_per_lap

0.08820000000000001

In [ ]:
dffp_soft['FuelGain'] = time_gained_per_lap
dffp_soft['FuelGain'] = dffp_soft['FuelGain'].cumsum()
dffp_soft['AdjustedTime'] = dffp_soft['LapTimeSeconds'] + dffp_soft['FuelGain']

In [ ]:
laptime_min_series = dffp_full[dffp_full['Compound'] == 'SOFT'].sort_values('LapTimeSeconds', ascending=True).iloc[0]
laptime_min = {'TyreLife': laptime_min_series['TyreLife'], 'LapTimeSeconds': laptime_min_series['LapTimeSeconds']}
laptime_min

{'TyreLife': 4.0, 'LapTimeSeconds': 91.20952173913044}

In [ ]:
laptime_min_calc_soft =  (laptime_min['LapTimeSeconds'] - ((laptime_min['TyreLife'] - 1) * diff_soft_per_lap)) + (nb_laps * time_gained_per_lap)
laptime_min_calc_soft

94.56387628458498

In [ ]:
laptime_min_calc_medium = (laptime_min_calc_soft + time_to_seconds(medium_to_soft))
laptime_min_calc_medium

95.35100309558497

In [ ]:
laptime_min_calc_hard = laptime_min_calc_soft + time_to_seconds(hard_to_soft)
laptime_min_calc_hard

96.47600798058498

In [ ]:
def get_deg_values(dc, dr):
    list_times = []

    for i in range(1, nb_laps + 1):
        dr = dr * (1 + dc)
        list_times.append(dr)
        
    return list_times

list_times_soft = get_deg_values(0.13, diff_soft_per_lap)
list_times_medium = get_deg_values(0.085, diff_medium_per_lap)
list_times_hard = get_deg_values(0.065, diff_medium_per_lap)

In [ ]:
df_times_soft = pd.DataFrame(columns=['Lap', 'Tyre', 'LapTimeSeconds', 'DeltaDeg', 'FuelDeg', 'AdjustedTime'])



df_times_soft['Lap'] = list(range(1, nb_laps + 1))
df_times_soft['Tyre'] = 'SOFT'
df_times_soft['LapTimeSeconds'] = laptime_min_calc_soft
df_times_soft['DeltaDeg'] = list_times_soft
df_times_soft['FuelDeg'] = time_gained_per_lap * df_times_soft['Lap']
df_times_soft['AdjustedTime'] = df_times_soft.apply(lambda row: row['LapTimeSeconds'] + row['DeltaDeg'], axis=1)
df_times_soft['FinalLapTime'] = df_times_soft.apply(lambda row: row['AdjustedTime'] - row['FuelDeg'], axis=1)


In [ ]:
df_times_medium = pd.DataFrame(columns=['Lap', 'Tyre', 'LapTimeSeconds', 'DeltaDeg', 'FuelDeg', 'AdjustedTime'])

df_times_medium['Lap'] = list(range(1, nb_laps + 1))
df_times_medium['Tyre'] = 'MEDIUM'
df_times_medium['LapTimeSeconds'] = laptime_min_calc_medium
df_times_medium['DeltaDeg'] = list_times_medium
df_times_medium['FuelDeg'] = time_gained_per_lap * df_times_medium['Lap']
df_times_medium['AdjustedTime'] = df_times_medium.apply(lambda row: row['LapTimeSeconds'] + row['DeltaDeg'], axis=1)
df_times_medium['FinalLapTime'] = df_times_medium.apply(lambda row: row['AdjustedTime'] - row['FuelDeg'], axis=1)


df_times_medium

,Lap,Tyre,LapTimeSeconds,DeltaDeg,FuelDeg,AdjustedTime,FinalLapTime
0,1,MEDIUM,95.351003,0.293480,0.0882,95.644483,95.556283
1,2,MEDIUM,95.351003,0.318425,0.1764,95.669428,95.493028
2,3,MEDIUM,95.351003,0.345492,0.2646,95.696495,95.431895
3,4,MEDIUM,95.351003,0.374858,0.3528,95.725861,95.373061
4,5,MEDIUM,95.351003,0.406721,0.4410,95.757724,95.316724
5,6,MEDIUM,95.351003,0.441293,0.5292,95.792296,95.263096
6,7,MEDIUM,95.351003,0.478802,0.6174,95.829806,95.212406
7,8,MEDIUM,95.351003,0.519501,0.7056,95.870504,95.164904
8,9,MEDIUM,95.351003,0.563658,0.7938,95.914661,95.120861
9,10,MEDIUM,95.351003,0.611569,0.8820,95.962572,95.080572


In [ ]:
df_times_hard = pd.DataFrame(columns=['Lap', 'Tyre', 'LapTimeSeconds', 'DeltaDeg', 'FuelDeg', 'AdjustedTime'])

df_times_hard['Lap'] = list(range(1, nb_laps + 1))
df_times_hard['Tyre'] = 'HARD'
df_times_hard['LapTimeSeconds'] = laptime_min_calc_hard
df_times_hard['DeltaDeg'] = list_times_hard
df_times_hard['FuelDeg'] = time_gained_per_lap * df_times_hard['Lap']
df_times_hard['AdjustedTime'] = df_times_hard.apply(lambda row: row['LapTimeSeconds'] + row['DeltaDeg'], axis=1)
df_times_hard['FinalLapTime'] = df_times_hard.apply(lambda row: row['AdjustedTime'] - row['FuelDeg'], axis=1)


df_times_hard

,Lap,Tyre,LapTimeSeconds,DeltaDeg,FuelDeg,AdjustedTime,FinalLapTime
0,1,HARD,96.476008,0.288070,0.0882,96.764078,96.675878
1,2,HARD,96.476008,0.306794,0.1764,96.782802,96.606402
2,3,HARD,96.476008,0.326736,0.2646,96.802744,96.538144
3,4,HARD,96.476008,0.347974,0.3528,96.823982,96.471182
4,5,HARD,96.476008,0.370592,0.4410,96.846600,96.405600
5,6,HARD,96.476008,0.394681,0.5292,96.870689,96.341489
6,7,HARD,96.476008,0.420335,0.6174,96.896343,96.278943
7,8,HARD,96.476008,0.447657,0.7056,96.923665,96.218065
8,9,HARD,96.476008,0.476754,0.7938,96.952762,96.158962
9,10,HARD,96.476008,0.507743,0.8820,96.983751,96.101751


In [ ]:
df_times = pd.concat([df_times_soft, df_times_medium, df_times_hard])
df_times['TimeStr'] = df_times['FinalLapTime'].apply(lambda x: str(datetime.timedelta(seconds=x))[2:-3])
df_times

,Lap,Tyre,LapTimeSeconds,DeltaDeg,FuelDeg,AdjustedTime,FinalLapTime,TimeStr
0,1,SOFT,94.563876,0.397626,0.0882,94.961503,94.873303,01:34.873
1,2,SOFT,94.563876,0.449318,0.1764,95.013194,94.836794,01:34.836
2,3,SOFT,94.563876,0.507729,0.2646,95.071606,94.807006,01:34.807
3,4,SOFT,94.563876,0.573734,0.3528,95.137610,94.784810,01:34.784
4,5,SOFT,94.563876,0.648319,0.4410,95.212196,94.771196,01:34.771
...,...,...,...,...,...,...,...,...
45,46,HARD,96.476008,4.900384,4.0572,101.376392,97.319192,01:37.319
46,47,HARD,96.476008,5.218909,4.1454,101.694917,97.549517,01:37.549
47,48,HARD,96.476008,5.558138,4.2336,102.034146,97.800546,01:37.800
48,49,HARD,96.476008,5.919417,4.3218,102.395425,98.073625,01:38.073


In [ ]:
hovertemplate = '<b>Lap:</b> %{x}<br><b>Time:</b> %{customdata}'

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_times[df_times['Tyre'] == 'SOFT']['Lap'],
                         y=df_times[df_times['Tyre'] == 'SOFT']['FinalLapTime'],
                         mode='lines',
                         name='Soft',
                         line_color=compound_colors['SOFT'],
                         customdata=df_times[df_times['Tyre'] == 'SOFT']['TimeStr'],
                         hovertemplate=hovertemplate))

fig.add_trace(go.Scatter(x=df_times[df_times['Tyre'] == 'MEDIUM']['Lap'],
                         y=df_times[df_times['Tyre'] == 'MEDIUM']['FinalLapTime'],
                         mode='lines',
                         name='Medium',
                         line_color=compound_colors['MEDIUM'],
                         customdata=df_times[df_times['Tyre'] == 'MEDIUM']['TimeStr'],
                         hovertemplate=hovertemplate))

fig.add_trace(go.Scatter(x=df_times[df_times['Tyre'] == 'HARD']['Lap'],
                         y=df_times[df_times['Tyre'] == 'HARD']['FinalLapTime'],
                         mode='lines',
                         name='Hard',
                         line_color=compound_colors['HARD'],
                         customdata=df_times[df_times['Tyre'] == 'HARD']['TimeStr'],
                         hovertemplate=hovertemplate))

fig.update_layout(width=1000, height=700, template='plotly_dark', yaxis_range=[df_times['FinalLapTime'].min() - 2, df_times['LapTimeSeconds'].max() + 10], hovermode='x')
fig.show()

In [ ]:
df_times.to_csv(f'../data/prediction_times_round_{gp_round}.csv')